In [58]:
import ssl

In [59]:
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonnasosauranga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Downloand git repositories automatically

In [60]:
from github import Github
import urllib.request
import zipfile
import io

# Access your GitHub account with a personal access token
g = Github('personal_access_token')

# Get a repository object for the repository you want to download
repo = g.get_repo('matplotlib/matplotlib')

# Download a ZIP archive of the repository
zipball_url = repo.get_archive_link('zipball')
response = urllib.request.urlopen(zipball_url)
data = response.read()

# Extract the files from the ZIP archive
with zipfile.ZipFile(io.BytesIO(data)) as zip_ref:
    zip_ref.extractall('./python-repositories')

"\n# Access your GitHub account with a personal access token\ng = Github('personal_access_token')\n\n# Get a repository object for the repository you want to download\nrepo = g.get_repo('username/repository')\n\n# Download a ZIP archive of the repository\nzipball_url = repo.get_archive_link('zipball')\nresponse = urllib.request.urlopen(zipball_url)\ndata = response.read()\n\n# Extract the files from the ZIP archive\nwith zipfile.ZipFile(io.BytesIO(data)) as zip_ref:\n    zip_ref.extractall('./python-repositories')"

# Clean repositories and move them to a new folder

In [61]:
#this piece of code filters the code files and cleans the non usefull files
import pathlib
import shutil
import os

def delete_subdirs(directory_path):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isdir(file_path):
            shutil.rmtree(file_path)
        else:
            os.remove(file_path)

# sets the path of the directory where the python files are downloades
path = pathlib.Path('./python-repositories')
#sets the file where we are moving the python files
new_dir = pathlib.Path('./python-files')

#loops trough all the the files in the directory
for p in path.rglob('*.py'):
    new_path = new_dir / p.name  # set the new file path
    p.rename(new_path) 

#removes all the folders with the non python files
path = pathlib.Path('./python-repositories/')
delete_subdirs(path)

# Ngram Model

In [65]:
import os
import tokenize
import string

# Define the path to the directory containing the Python files
dir_path = './python-files'

# Create an empty list to store the code from all Python files
code_list = []

# Loop through each file in the directory and extract the code
for filename in os.listdir(dir_path):
    if filename.endswith('.py'):
        try:
            with open(os.path.join(dir_path, filename), 'r') as f:
                text = f.read()  # read the entire file as a single string
                code_list.append(text)
        except:
            pass
        
# Concatenate the code from all files into a single string
code = '\n'.join(code_list)

# Count the number of lines of code
num_lines = code.count('\n')

# Tokenize the code by lowercasing everything and splitting on all whitespace and punctuation characters

tokens = []
for token in tokenize.tokenize(io.BytesIO(code.encode('utf-8')).readline):
    if token.type == tokenize.NAME:
        # Remove any leading or trailing spaces and punctuation from the token
        token_value = token.string.lower().strip(string.whitespace + string.punctuation)
        if token_value:
            tokens.append(token_value)

# Count the number of tokens
num_tokens = len(tokens)

In [66]:
# ngram dictionary to keep the possible ngrams as keys and next occuring words as list items
ngrams = {}

# words in a single ngram
words = 3

# Iterate over words tokens
for i in range(len(tokens)-words):
    # incrementaly find ngram word sequences  
    seq = ' '.join(tokens[i:i+words])

    # insert in dictionary as key
    if  seq not in ngrams.keys():
        # prepare empty list to insert next possible chars
        ngrams[seq] = []

    # push a next word
    ngrams[seq].append(tokens[i+words])


In [67]:
import random

def auto_complete(seed_sequence):
    # assuming very first ngram as a search sequence
    search_sequence = seed_sequence #' '.join(words_tokens[0:words])

    # init the suggestion output
    output = search_sequence

    next_word = ""

    # lenth of max words in obtained suggestion
    #while next_word != ".":
    for i in range(10):
        # break, if search sequence is not present in prepared ngram dictionary
        if search_sequence not in ngrams.keys():
            search_sequence = random.choice(list(ngrams.keys()))
            #print(search_sequence)
        
        # if ngram key is available, then find the list of next possible characters
        possible_words = ngrams[search_sequence]

        # Randomly select the next possbile character, most common will have more chances
        next_word = possible_words[random.randrange(len(possible_words))]

        # Update the suggestion output
        output += ' ' + next_word
        seq_words = nltk.word_tokenize(output)

        # update the search sequence now, excluding first char as we move forward
        search_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])
        
    return output


In [68]:
seed_sequence = "test pandas"
print(f'Search sequence: ', seed_sequence)
print(f'\nSuggestion: {auto_complete(seed_sequence)}')

Search sequence:  test pandas

Suggestion: test pandas self parametrize self test_demo testcase len self path vertices if


# Gensim Word2Vec model

In [ ]:
import os
import tokenize
import string

# Define the path to the directory containing the Python files
dir_path = './python-files'

# Create an empty list to store the code from all Python files
tokenized_lines_of_code = []

for filename in os.listdir(dir_path):
    if filename.endswith('.py'):
        try:
            with open(os.path.join(dir_path, filename), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    tokens = []
                    for token in tokenize.tokenize(io.BytesIO(line.encode('utf-8')).readline):
                        if token.type == tokenize.NAME:
                            # Remove any leading or trailing spaces and punctuation from the token
                            token_value = token.string.lower().strip(string.whitespace + string.punctuation)
                            if token_value:
                                tokens.append(token_value)
                    tokenized_lines_of_code.append(tokens)
        except:
            pass

In [ ]:
#created the Word2Vec model

import gensim

model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

model.build_vocab(tokenized_lines_of_code)
model.train(tokenized_lines_of_code, total_examples=model.corpus_count, epochs=model.epochs)

(595632, 894270)

In [ ]:
model.wv.most_similar('if')

[('and', 0.9747416973114014),
 ('not', 0.964296281337738),
 ('hasattr', 0.940148651599884),
 ('startswith', 0.9391381144523621),
 ('or', 0.93909752368927),
 ('endswith', 0.9301563501358032),
 ('l', 0.9260615706443787),
 ('line', 0.9229528307914734),
 ('poll', 0.9222391843795776),
 ('container_name', 0.9213384389877319)]

In [ ]:
model.wv.most_similar('and')

[('startswith', 0.9870865345001221),
 ('hasattr', 0.9851466417312622),
 ('if', 0.9747416973114014),
 ('endswith', 0.9738984704017639),
 ('not', 0.9726549386978149),
 ('requirements', 0.9637553691864014),
 ('items', 0.9629218578338623),
 ('line', 0.9614998698234558),
 ('found', 0.9612892270088196),
 ('container_name', 0.9610879421234131)]

In [ ]:
model.wv.most_similar('pandas')

[('test_decorators', 0.9226421117782593),
 ('dtypes', 0.9109344482421875),
 ('util', 0.9021406173706055),
 ('libs', 0.8971794247627258),
 ('td', 0.890876054763794),
 ('internals', 0.8865272402763367),
 ('gettext', 0.8861660957336426),
 ('find_spec', 0.878078281879425),
 ('matplotlib', 0.8744655251502991),
 ('interchange', 0.8726103901863098)]